## Traning

In [1]:
!git clone https://github.com/iiscleap/Coswara-Data.git

Cloning into 'Coswara-Data'...
remote: Enumerating objects: 995, done.
remote: Counting objects: 100% (197/197), done.
remote: Compressing objects: 100% (115/115), done.
remote: Total 995 (delta 112), reused 148 (delta 77), pack-reused 798
Receiving objects: 100% (995/995), 15.97 GiB | 38.10 MiB/s, done.
Resolving deltas: 100% (422/422), done.
Updating files: 100% (336/336), done.


In [2]:
import os
import subprocess
import glob

coswara_data_dir = os.path.abspath('/content/Coswara-Data')  # Local Path of iiscleap/Coswara-Data Repo
extracted_data_dir = os.path.join(coswara_data_dir, 'Extracted_data')

if not os.path.exists(coswara_data_dir):
    raise ValueError("Check the Coswara dataset directory!")

if not os.path.exists(extracted_data_dir):
    os.makedirs(extracted_data_dir)  # Creates the Extracted_data folder if it doesn't exist

dirs_extracted = set(map(os.path.basename, glob.glob('{}/202*'.format(extracted_data_dir))))
dirs_all = set(map(os.path.basename, glob.glob('{}/202*'.format(coswara_data_dir))))

dirs_to_extract = list(set(dirs_all) - dirs_extracted)

for d in dirs_to_extract:
    command = 'cat {0}/{1}/*.tar.gz.* | tar -xvz -C {2}'.format(coswara_data_dir, d, extracted_data_dir)
    subprocess.run(command, shell=True, check=True)

print("Extraction process complete!")


Extraction process complete!


In [3]:
import os
import shutil

# Set the path to the root folder
root_folder = '/content/Coswara-Data/Extracted_data'  # Replace with the actual path

# Set the path to the target folder
target_folder = os.path.join(root_folder, 'audio_data')

# Create the target folder if it doesn't exist
if not os.path.exists(target_folder):
    os.makedirs(target_folder)

# List all folders in the root directory
all_folders = os.listdir(root_folder)

# Iterate through folders starting with "2"
for folder in all_folders:
    if folder.startswith("2") and os.path.isdir(os.path.join(root_folder, folder)):
        # Get the path of the source folder
        source_folder = os.path.join(root_folder, folder)

        # Move the contents of the source folder to the target folder
        for item in os.listdir(source_folder):
            source_item = os.path.join(source_folder, item)
            target_item = os.path.join(target_folder, item)
            shutil.move(source_item, target_item)

        # Remove the empty source folder
        os.rmdir(source_folder)

print("Move operation complete.")


Move operation complete.


In [4]:
import os
import pandas as pd
import numpy as np
import soundfile as sf
import librosa

In [5]:
df = pd.read_csv('/content/Coswara-Data/combined_data.csv')

In [6]:
def encoder(row):
    if row['covid_status'].startswith('p'):
        return 1
    else:
        for x in ['asthma', 'cough', 'smoker', 'ht', 'cold', 'diabetes', 'diarrhoea', 'um', 'ihd', 'bd', 'st', 'fever', 'ftg', 'mp', 'loss_of_smell', 'cld', 'pneumonia']:
            if str(row[x]).lower() == 'true' or str(row[x]).lower() == 'false':
                return 1
        return 0

# Assuming df is your DataFrame
df['label'] = df.apply(encoder, axis=1)

In [7]:
newdf = df[['id', 'label']].copy()

In [8]:



# Process each row in the DataFrame
paths_list = []
for index, row in newdf.iterrows():
    folder_name = row['id']

    # List of audio files for the current folder
    audio_files = [
        'breathing-deep.wav',
        'breathing-shallow.wav',
        'cough-heavy.wav',
        'cough-shallow.wav',
        'counting-fast.wav',
        'counting-normal.wav',
        'vowel-a.wav',
        'vowel-e.wav',
        'vowel-o.wav'
    ]

    # Create the array of full paths for the current folder
    full_paths = [os.path.join(f'/content/Coswara-Data/Extracted_data/audio_data/{folder_name}', file) for file in audio_files]
    paths_list.append(full_paths)

# Display the resulting list of paths for each folder
print(paths_list)


Output hidden; open in https://colab.research.google.com to view.

In [9]:
# Add the 'full_paths' column to the DataFrame
newdf['full_paths'] = paths_list

In [ ]:
newdf.head()

,id,label,full_paths,audio_paths,features
0,iV3Db6t1T8b7c5HQY2TwxIhjbzD3,0,[/content/Coswara-Data/Extracted_data/audio_da...,001Output_combined_audio\/combined_audio_iV3Db...,None
1,AxuYWBN0jFVLINCBqIW5aZmGCdu1,1,[/content/Coswara-Data/Extracted_data/audio_da...,001Output_combined_audio\/combined_audio_AxuYW...,None
2,C5eIsssb9GSkaAgIfsHMHeR6fSh1,0,[/content/Coswara-Data/Extracted_data/audio_da...,001Output_combined_audio\/combined_audio_C5eIs...,None
3,YjbEAECMBIaZKyfqOvWy5DDImUb2,1,[/content/Coswara-Data/Extracted_data/audio_da...,001Output_combined_audio\/combined_audio_YjbEA...,None
4,aGOvk4ji0cVqIzCs1jHnzlw2UEy2,0,[/content/Coswara-Data/Extracted_data/audio_da...,001Output_combined_audio\/combined_audio_aGOvk...,None


In [15]:
import os
import pandas as pd
import numpy as np
import soundfile as sf
import librosa

# Function to concatenate specified audio files into a single file
def concatenate_audio_files(input_files, output_file):
    try:
        combined_signal, combined_sr = librosa.load(input_files[0], sr=None)
        for file_path in input_files[1:]:
            signal, sr = librosa.load(file_path, sr=None)
            combined_signal = np.concatenate([combined_signal, signal])
    except Exception as e:
        print(f"Error encountered while loading audio files: {e}")
        return None

    # Save the combined audio file using soundfile
    try:
        sf.write(output_file, combined_signal, combined_sr)
    except Exception as e:
        print(f"Error encountered while saving combined audio file: {e}")
        return None

    return output_file


# Output folder for combined audio files
output_folder = '/content/001Output_combined_audio'
os.makedirs(output_folder, exist_ok=True)

# Process each row in the DataFrame
for index, row in newdf.iterrows():
    folder_name = row['id']
    output_file = os.path.join(output_folder, f"combined_audio_{folder_name}.wav")

    # List of audio files for the current folder
    audio_files = row['full_paths']

    # Concatenate all audio files into a single file for each folder
    result = concatenate_audio_files(audio_files, output_file)

    if result:
        continue
    else:
        print(f"Error concatenating audio files for folder {folder_name}")


<ipython-input-15-cadc2f04d28f>:12: UserWarning: PySoundFile failed. Trying audioread instead.
  signal, sr = librosa.load(file_path, sr=None)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Error encountered while loading audio files: [Errno 2] No such file or directory: '/content/Coswara-Data/Extracted_data/audio_data/o0HUIrKBsMXkygZgI161LVIydIp1/vowel-o.wav'
Error concatenating audio files for folder o0HUIrKBsMXkygZgI161LVIydIp1
Error encountered while loading audio files: [Errno 2] No such file or directory: '/content/Coswara-Data/Extracted_data/audio_data/9hftEYixyhP1Neeq3fB7ZwITQC53/cough-shallow.wav'
Error concatenating audio files for folder 9hftEYixyhP1Neeq3fB7ZwITQC53


In [13]:
# New path prefix for combined audio files
new_path_prefix = '/content/001Output_combined_audio'

# Function to update paths
def update_path(row):
    return os.path.join(new_path_prefix, f"combined_audio_{row['id']}.wav")

# Apply the function to update paths
newdf['audio_paths'] = newdf.apply(update_path, axis=1)


In [16]:
import pandas as pd
import librosa
import numpy as np

def extract_audio_features(audio_path):
    try:
        y, sr = librosa.load(audio_path)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        rmse = librosa.feature.rms(y=y)
        spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
        spectral_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        features = np.hstack([chroma_stft.mean(axis=1), rmse.mean(), spectral_centroid.mean(), spectral_bandwidth.mean()])
        return features
    except Exception as e:
        print(f"Error processing {audio_path}: {str(e)}")
        return None  # Set feature value to None for paths that could not be processed


# Create a new column 'features' to store the extracted features
newdf['features'] = newdf['audio_paths'].apply(extract_audio_features)


/usr/local/lib/python3.10/dist-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(
/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(
<ipython-input-16-2ae34b24e27b>:7: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(audio_path)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Error processing /content/001Output_combined_audio/combined_audio_o0HUIrKBsMXkygZgI161LVIydIp1.wav: [Errno 2] No such file or directory: '/content/001Output_combined_audio/combined_audio_o0HUIrKBsMXkygZgI161LVIydIp1.wav'
Error processing /content/001Output_combined_audio/combined_audio_9hftEYixyhP1Neeq3fB7ZwITQC53.wav: [Errno 2] No such file or directory: '/content/001Output_combined_audio/combined_audio_9hftEYixyhP1Neeq3fB7ZwITQC53.wav'


In [17]:
newdf

,id,label,full_paths,audio_paths,features
0,iV3Db6t1T8b7c5HQY2TwxIhjbzD3,0,[/content/Coswara-Data/Extracted_data/audio_da...,/content/001Output_combined_audio/combined_aud...,"[0.16860748827457428, 0.15911227464675903, 0.1..."
1,AxuYWBN0jFVLINCBqIW5aZmGCdu1,1,[/content/Coswara-Data/Extracted_data/audio_da...,/content/001Output_combined_audio/combined_aud...,"[0.4609575867652893, 0.43077656626701355, 0.31..."
2,C5eIsssb9GSkaAgIfsHMHeR6fSh1,0,[/content/Coswara-Data/Extracted_data/audio_da...,/content/001Output_combined_audio/combined_aud...,"[0.39514949917793274, 0.3208620846271515, 0.33..."
3,YjbEAECMBIaZKyfqOvWy5DDImUb2,1,[/content/Coswara-Data/Extracted_data/audio_da...,/content/001Output_combined_audio/combined_aud...,"[0.34726041555404663, 0.35330966114997864, 0.3..."
4,aGOvk4ji0cVqIzCs1jHnzlw2UEy2,0,[/content/Coswara-Data/Extracted_data/audio_da...,/content/001Output_combined_audio/combined_aud...,"[0.45289701223373413, 0.4587215781211853, 0.44..."
...,...,...,...,...,...
2741,333NjqA1TfZJuICEdXSkPhVz0LA3,1,[/content/Coswara-Data/Extracted_data/audio_da...,/content/001Output_combined_audio/combined_aud...,"[0.35809123516082764, 0.2891637682914734, 0.28..."
2742,dpA0EeRrtJUeKJjEuf7BL0AeTJZ2,1,[/content/Coswara-Data/Extracted_data/audio_da...,/content/001Output_combined_audio/combined_aud...,"[0.3886304795742035, 0.39853429794311523, 0.40..."
2743,UXhG3vgoxkWtd4Meky1nm0sRgMV2,1,[/content/Coswara-Data/Extracted_data/audio_da...,/content/001Output_combined_audio/combined_aud...,"[0.29401353001594543, 0.31097647547721863, 0.3..."
2744,A5KW9PXraNgckln1gnNaJnX6DvB2,1,[/content/Coswara-Data/Extracted_data/audio_da...,/content/001Output_combined_audio/combined_aud...,"[0.47903528809547424, 0.48960134387016296, 0.4..."


In [18]:
# Assuming your DataFrame is named 'newdf'
newdf_cleaned = newdf.dropna(subset=['features'])

In [19]:
newdf_cleaned

,id,label,full_paths,audio_paths,features
0,iV3Db6t1T8b7c5HQY2TwxIhjbzD3,0,[/content/Coswara-Data/Extracted_data/audio_da...,/content/001Output_combined_audio/combined_aud...,"[0.16860748827457428, 0.15911227464675903, 0.1..."
1,AxuYWBN0jFVLINCBqIW5aZmGCdu1,1,[/content/Coswara-Data/Extracted_data/audio_da...,/content/001Output_combined_audio/combined_aud...,"[0.4609575867652893, 0.43077656626701355, 0.31..."
2,C5eIsssb9GSkaAgIfsHMHeR6fSh1,0,[/content/Coswara-Data/Extracted_data/audio_da...,/content/001Output_combined_audio/combined_aud...,"[0.39514949917793274, 0.3208620846271515, 0.33..."
3,YjbEAECMBIaZKyfqOvWy5DDImUb2,1,[/content/Coswara-Data/Extracted_data/audio_da...,/content/001Output_combined_audio/combined_aud...,"[0.34726041555404663, 0.35330966114997864, 0.3..."
4,aGOvk4ji0cVqIzCs1jHnzlw2UEy2,0,[/content/Coswara-Data/Extracted_data/audio_da...,/content/001Output_combined_audio/combined_aud...,"[0.45289701223373413, 0.4587215781211853, 0.44..."
...,...,...,...,...,...
2741,333NjqA1TfZJuICEdXSkPhVz0LA3,1,[/content/Coswara-Data/Extracted_data/audio_da...,/content/001Output_combined_audio/combined_aud...,"[0.35809123516082764, 0.2891637682914734, 0.28..."
2742,dpA0EeRrtJUeKJjEuf7BL0AeTJZ2,1,[/content/Coswara-Data/Extracted_data/audio_da...,/content/001Output_combined_audio/combined_aud...,"[0.3886304795742035, 0.39853429794311523, 0.40..."
2743,UXhG3vgoxkWtd4Meky1nm0sRgMV2,1,[/content/Coswara-Data/Extracted_data/audio_da...,/content/001Output_combined_audio/combined_aud...,"[0.29401353001594543, 0.31097647547721863, 0.3..."
2744,A5KW9PXraNgckln1gnNaJnX6DvB2,1,[/content/Coswara-Data/Extracted_data/audio_da...,/content/001Output_combined_audio/combined_aud...,"[0.47903528809547424, 0.48960134387016296, 0.4..."


In [21]:
a = newdf_cleaned['features'][0]

In [22]:
a.shape

(15,)

In [20]:
# Save the cleaned DataFrame to a new CSV file
newdf_cleaned.to_csv('cleaned_data(RESPERA).csv', index=False)

In [58]:
from sklearn.model_selection import train_test_split

# Extract features and labels
features = np.array(list(newdf_cleaned['features']))
labels = np.array(list(newdf_cleaned['label']))

# Split the data into training, validation, and test sets
X_train, X_temp, Y_train, Y_temp = train_test_split(features, labels, test_size=0.3, random_state=42)
X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=42)

# Display the shapes
print(f"X_train shape: {X_train.shape}, Y_train shape: {Y_train.shape}")
print(f"X_val shape: {X_val.shape}, Y_val shape: {Y_val.shape}")
print(f"X_test shape: {X_test.shape}, Y_test shape: {Y_test.shape}")


X_train shape: (1920, 15), Y_train shape: (1920,)
X_val shape: (412, 15), Y_val shape: (412,)
X_test shape: (412, 15), Y_test shape: (412,)


create_audio_classification_model

In [29]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the model
def create_audio_classification_model(input_shape, num_classes):
    model = models.Sequential()
    model.add(layers.InputLayer(input_shape=input_shape))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(num_classes, activation='softmax'))
    return model

# Set input shape and number of classes based on your data
input_shape = (15,)  # Update with the actual length of your feature vectors
num_classes = 2  # Assuming binary classification (0 and 1)

# Create the model
model = create_audio_classification_model(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()

# Train the model
history = model.fit(X_train, Y_train, epochs=10, validation_data=(X_val, Y_val))

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, Y_test)
print(f"Test accuracy: {test_acc}")


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 128)               2048      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_4 (Dense)             (None, 2)                 130       
                                                                 
Total params: 10434 (40.76 KB)
Trainable params: 10434 (40.76 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
60/60 [

In [36]:
# Save the previous model
model.save("create_audio_classification_model.h5")


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Cnn Model

In [35]:
from tensorflow.keras import layers, models

# Define the CNN model
def create_cnn_model(input_shape, num_classes, learning_rate=0.001):
    model = models.Sequential()

    # Reshape the input to have a single channel
    model.add(layers.Reshape((input_shape[0], 1), input_shape=input_shape))

    # Convolutional layers
    model.add(layers.Conv1D(64, kernel_size=5, activation='relu'))
    model.add(layers.MaxPooling1D(pool_size=2, padding='same'))
    model.add(layers.Conv1D(128, kernel_size=5, activation='relu'))
    model.add(layers.MaxPooling1D(pool_size=2, padding='same'))
    model.add(layers.Conv1D(256, kernel_size=5, activation='relu'))

    # Global Max Pooling
    model.add(layers.GlobalMaxPooling1D())

    # Fully connected layers
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.3))

    # Output layer with softmax activation for multi-class classification
    model.add(layers.Dense(num_classes, activation='softmax'))

    # Compile the model with the specified learning rate
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# Set a minimum input size to avoid downsampling issues
min_input_size = 50

# Set input shape and number of classes based on your data
input_shape = (max(min_input_size, 15), )
num_classes = 2




Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_4 (Reshape)         (None, 50, 1)             0         
                                                                 
 conv1d_11 (Conv1D)          (None, 46, 64)            384       
                                                                 
 max_pooling1d_8 (MaxPoolin  (None, 23, 64)            0         
 g1D)                                                            
                                                                 
 conv1d_12 (Conv1D)          (None, 19, 128)           41088     
                                                                 
 max_pooling1d_9 (MaxPoolin  (None, 10, 128)           0         
 g1D)                                                            
                                                                 
 conv1d_13 (Conv1D)          (None, 6, 256)           

In [41]:
# Define the desired sequence length
sequence_length = 50

# Determine the size of the first dimension after reshaping
num_samples_train = len(X_train) // sequence_length
num_samples_test = len(X_test) // sequence_length

# Reshape and pad the training data
X_train_processed = X_train[:num_samples_train * sequence_length]
X_train_processed = np.pad(X_train_processed, ((0, sequence_length - len(X_train_processed) % sequence_length), (0, 0)))
X_train_processed = X_train_processed.reshape(-1, sequence_length, 1)

# Reshape and pad the test data
X_test_processed = X_test[:num_samples_test * sequence_length]
X_test_processed = np.pad(X_test_processed, ((0, sequence_length - len(X_test_processed) % sequence_length), (0, 0)))
X_test_processed = X_test_processed.reshape(-1, sequence_length, 1)



# Create the CNN model
cnn_model = create_cnn_model((50, 1), num_classes, learning_rate=0.0001)

# Compile the model
cnn_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

# Display the model summary
cnn_model.summary()

# Train the model with the processed data
history = cnn_model.fit(X_train_processed, Y_train, epochs=10, validation_split=0.2, batch_size=32)

# Evaluate the model on the processed test set
test_loss, test_acc = cnn_model.evaluate(X_test_processed, Y_test)


Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_5 (Reshape)         (None, 50, 1)             0         
                                                                 
 conv1d_14 (Conv1D)          (None, 46, 64)            384       
                                                                 
 max_pooling1d_10 (MaxPooli  (None, 23, 64)            0         
 ng1D)                                                           
                                                                 
 conv1d_15 (Conv1D)          (None, 19, 128)           41088     
                                                                 
 max_pooling1d_11 (MaxPooli  (None, 10, 128)           0         
 ng1D)                                                           
                                                                 
 conv1d_16 (Conv1D)          (None, 6, 256)           

ValueError: Data cardinality is ambiguous:
  x sizes: 135
  y sizes: 412
Make sure all arrays contain the same number of samples.

In [42]:
# Assuming X_test is your original test set
# Reshape the test data to match the expected input shape of the model
X_test_processed = X_test.reshape(-1, 50, 1)

# Evaluate the model on the processed test set
test_loss, test_acc = cnn_model.evaluate(X_test_processed, Y_test)
print(f"Test accuracy: {test_acc}")


ValueError: cannot reshape array of size 6180 into shape (50,1)

In [ ]:
cnn_model.save("CnnModel.Keras")

In [ ]:
import joblib

# Save the trained model to a file
model_filename = 'RESPERA-0.1.joblib'
joblib.dump(model, model_filename)

['RESPERA-0.1.joblib']

RNN MODEL

In [54]:
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense, Dropout, Reshape
from keras.optimizers import Adam

# Function to create RNN model
def create_rnn_model(input_shape, num_classes, learning_rate=0.001):
    model = Sequential()

    # Reshape the input data
    model.add(Reshape((input_shape[0], 1), input_shape=input_shape))

    # RNN layer
    model.add(SimpleRNN(64, activation='relu', return_sequences=True))
    model.add(SimpleRNN(64, activation='relu'))

    # Dense layers
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))

    # Output layer
    model.add(Dense(num_classes, activation='softmax'))

    # Compile the model
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])

    return model

# Assuming input_shape is (15,) and num_classes is 2
input_shape_rnn = (15,)
num_classes_rnn = 2

# Create the RNN model
rnn_model = create_rnn_model(input_shape_rnn, num_classes_rnn, learning_rate=0.00001)

# Display the model summary
rnn_model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_11 (Reshape)        (None, 15, 1)             0         
                                                                 
 simple_rnn_9 (SimpleRNN)    (None, 15, 64)            4224      
                                                                 
 simple_rnn_10 (SimpleRNN)   (None, 64)                8256      
                                                                 
 dense_19 (Dense)            (None, 128)               8320      
                                                                 
 dropout_10 (Dropout)        (None, 128)               0         
                                                                 
 dense_20 (Dense)            (None, 2)                 258       
                                                                 
Total params: 21058 (82.26 KB)
Trainable params: 2105

In [55]:
# Assuming you've already created and compiled your RNN model
# (replace `rnn_model` with your actual model instance)

# Train the RNN model
history = rnn_model.fit(
    X_train, Y_train,
    epochs=10,
    validation_data=(X_val, Y_val),
    batch_size=32
)

# Evaluate the RNN model on the test set
test_loss, test_acc = rnn_model.evaluate(X_test, Y_test)
print(f'Test accuracy: {test_acc * 100:.2f}%')


Epoch 1/10
60/60 [==============================] - 3s 24ms/step - loss: 118.5929 - accuracy: 0.5021 - val_loss: 45.7052 - val_accuracy: 0.5316
Epoch 2/10
60/60 [==============================] - 1s 17ms/step - loss: 104.8806 - accuracy: 0.4938 - val_loss: 23.3108 - val_accuracy: 0.5316
Epoch 3/10
60/60 [==============================] - 1s 11ms/step - loss: 96.4372 - accuracy: 0.5047 - val_loss: 14.7834 - val_accuracy: 0.5316
Epoch 4/10
60/60 [==============================] - 1s 9ms/step - loss: 99.8336 - accuracy: 0.4958 - val_loss: 6.9256 - val_accuracy: 0.5316
Epoch 5/10
60/60 [==============================] - 1s 9ms/step - loss: 96.2265 - accuracy: 0.4802 - val_loss: 4.9381 - val_accuracy: 0.5243
Epoch 6/10
60/60 [==============================] - 1s 10ms/step - loss: 82.7526 - accuracy: 0.5021 - val_loss: 3.1699 - val_accuracy: 0.5437
Epoch 7/10
60/60 [==============================] - 1s 9ms/step - loss: 76.2819 - accuracy: 0.5078 - val_loss: 3.1914 - val_accuracy: 0.5437
Epoc

In [89]:
rnn_model.save("Resperia.keras")

In [90]:
!cp /content/Resperia.keras /content/drive/MyDrive

SKlearn

In [59]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report


# Initialize the RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
model.fit(X_train, Y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(Y_test, y_pred)
classification_report_str = classification_report(Y_test, y_pred)

# Print the evaluation metrics
print(f'Accuracy: {accuracy}')
print('Classification Report:\n', classification_report_str)


Accuracy: 0.5946601941747572
Classification Report:
               precision    recall  f1-score   support

           0       0.53      0.57      0.55       180
           1       0.65      0.62      0.63       232

    accuracy                           0.59       412
   macro avg       0.59      0.59      0.59       412
weighted avg       0.60      0.59      0.60       412



##App

In [73]:
import numpy as np
import glob
import os
import librosa
import soundfile as sf
import time
import keras

print('Loading Model....')
# Load the trained Keras model - create_audio_classification_model.h5
model_filename = 'create_audio_classification_model.h5'
trained_model = keras.models.load_model(model_filename)
print('Model Loaded....')


def get_wav_files_in_folder(folder_path):
    # Construct the search pattern for WAV files
    search_pattern = os.path.join(folder_path, '*.wav')

    # Use glob to get a list of paths matching the pattern
    wav_files = glob.glob(search_pattern)

    return wav_files



def concatenate_audio_files(input_files):
    try:
        combined_signal, combined_sr = librosa.load(input_files[0], sr=None)
        for file_path in input_files[1:]:
            signal, sr = librosa.load(file_path, sr=None)
            combined_signal = np.concatenate([combined_signal, signal])
    except Exception as e:
        print(f"Error encountered while loading audio files: {e}")
        return None, None

    return combined_signal, combined_sr


def extract_audio_features(audio_signal, sr):
    try:
        chroma_stft = librosa.feature.chroma_stft(y=audio_signal, sr=sr)
        rmse = librosa.feature.rms(y=audio_signal)
        spectral_centroid = librosa.feature.spectral_centroid(y=audio_signal, sr=sr)
        spectral_bandwidth = librosa.feature.spectral_bandwidth(y=audio_signal, sr=sr)
        features = np.hstack([chroma_stft.mean(axis=1), rmse.mean(), spectral_centroid.mean(), spectral_bandwidth.mean()])
        return features
    except Exception as e:
        print(f"Error processing audio signal: {str(e)}")
        return None


print('Loading Audio Files....')
file = input('Enter Path: ')
paths = get_wav_files_in_folder(file)
# Load the combined audio file and extract features
combined_audio, sr = concatenate_audio_files(paths)

# Check if loading and concatenation were successful
if combined_audio is not None:
    print('Extracting Audio Features....')
    # Extract features from the combined audio
    audio_features = extract_audio_features(combined_audio,sr)

    if audio_features is not None:
        # Reshape the features to match the model's expected input shape
        audio_features_reshaped = audio_features.reshape(1, -1)
        print('Predicting....')
        time.sleep(5)
        # Make predictions using the trained model
        prediction = trained_model.predict(audio_features_reshaped)

        if prediction is not None:
            print(f"The predicted label for the audio file is: {prediction}")
            print('\n\n\n\n')
            print(f"""            *****************************************************
            Probabilty of not having respiratory disease {prediction[0][0]*100:.4f}
            Probabilty of having respiratory disease {prediction[0][1]*100:.4f}
            *********************************************************""")
            print('\n\n\n\n')
        else:
            print('Failed')
    else:
        print('Error extracting audio features.')
else:
    print('Error loading or concatenating audio files.')



Loading Model....
Model Loaded....
Loading Audio Files....
Enter Path: /content/Coswara-Data/Extracted_data/audio_data/0EAAFsDWfTcrhktHy78LS6nf19G3
Extracting Audio Features....
Predicting....
1/1 [==============================] - 0s 66ms/step
The predicted label for the audio file is: [[0.41379455 0.58620554]]





            *****************************************************
            Probabilty of not having respiratory disease 41.3795
            Probabilty of having respiratory disease 58.6206
            *********************************************************







In [74]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [88]:
!mv /content/Audio_Sample /content/drive/MyDrive

In [79]:
!cd /content/Coswara-Data/Extracted_data/audio_data/00xKcQMmcAhX8CODgBBLOe7Dm0T2 to /content/sample_data

/bin/bash: line 1: cd: too many arguments


In [94]:
# Move the directory to /content/sample_data
!mv /content/Coswara-Data/Extracted_data/audio_data/GjdjAvKrnbRRRX1KbioknjMVLNx2 /content/drive/MyDrive/Audio_Sample